In [1]:
import numpy as np
import matplotlib.pylab as plt

In [2]:
from pyiron import Project
from pyiron.table.datamining import PyironTable

In [3]:
pr = Project("Al_murnaghan_automated")

In [4]:
pt = PyironTable(pr, "murn_table")

In [5]:
def filter_function(job):
    return (job.status == "finished") & ("murn" in job.job_name)

In [6]:
pt.filter_function = filter_function

In [7]:
def get_lattice_parameter(job):
    return job["output/equilibrium_volume"] ** (1/3)

def get_bm(job):
    return job["output/equilibrium_bulk_modulus"]

def get_pot(job):
    child = job.project.inspect(job["output/id"][0])
    return child["input/potential/Name"]

def get_bulk_energy_size(job, size=4):
    a = get_lattice_parameter(job)
    el = job["output/structure/species"][0]
    ref_supercell = pr.create_ase_bulk(el, a=a).repeat(size)
    pot = get_pot(job)
    ref_job_name = "rj_{}_s_{}".format(pot, size).replace("-", "_")
    df = pr.job_table()
    if not ref_job_name in df[df.status=="finished"].job.to_list():
        ref_job = pr.create_job(pr.job_type.Lammps, ref_job_name)
        ref_job.structure = ref_supercell
        ref_job.potential = pot
        ref_job.calc_minimize()
        ref_job.run()
    e_ref = pr.inspect(ref_job_name)["output/generic/energy_tot"][-1]
    n_ref = pr.inspect(ref_job_name)["output/generic/positions"].shape[1]
    return e_ref, n_ref 

def get_vac_formation_energy(job, size=4):
    a = get_lattice_parameter(job)
    el = job["output/structure/species"][0]
    ref_supercell = pr.create_ase_bulk(el, a=a).repeat(size)
    def_supercell = ref_supercell[0:-1]
    pot = get_pot(job)
    ref_job_name = "rj_{}_s_{}".format(pot, size).replace("-", "_")
    def_job_name = "dj_{}_s_{}".format(pot, size).replace("-", "_")
    df = pr.job_table()
    if not ref_job_name in df[df.status=="finished"].job.to_list():
        ref_job = pr.create_job(pr.job_type.Lammps, ref_job_name)
        ref_job.structure = ref_supercell
        ref_job.potential = pot
        ref_job.calc_minimize()
        ref_job.run()
    e_ref = pr.inspect(ref_job_name)["output/generic/energy_tot"][-1]
    n_ref = pr.inspect(ref_job_name)["output/generic/positions"].shape[1]
    if not def_job_name in df[df.status=="finished"].job.to_list():
        def_job = pr.create_job(pr.job_type.Lammps, def_job_name)
        def_job.structure = def_supercell
        def_job.potential = pot
        def_job.calc_minimize()
        def_job.run()
    e_def = pr.inspect(def_job_name)["output/generic/energy_tot"][-1]
    n_def = pr.inspect(def_job_name)["output/generic/positions"].shape[1]
    return (e_def - e_ref * (n_def/n_ref))

def get_cohesive_energy(job, size=4):
    e_bulk, n_bulk = get_bulk_energy_size(job, size)
    pot = get_pot(job)
    atom_job_name = "aj_{}_s_{}".format(pot, size).replace("-", "_")
    df = pr.job_table()
    if not atom_job_name in df[df.status=="finished"].job.to_list():
        atom_job = pr.create_job(pr.job_type.Lammps, atom_job_name)
        el = job["output/structure/species"][0]
        atom_job.structure = pr.create_atoms(el, cell=np.eye(3)* 20, scaled_positions=[[0.5, 0.5, 0.5]])
        atom_job.potential = pot
        atom_job.calc_static()
        atom_job.run()
    e_atom = pr.inspect(atom_job_name)["output/generic/energy_tot"][-1]
    return e_bulk / n_bulk - e_atom


In [8]:
pt.add["a_eq"] = get_lattice_parameter
pt.add["bulk_modulus"] = get_bm
pt.add["potential"] = get_pot
pt.add["vac_formation"] = get_vac_formation_energy
pt.add["ecoh"] = get_cohesive_energy

In [9]:
pt.convert_to_object = False

In [10]:
pt.create_table()

100%|██████████| 37/37 [00:12<00:00,  3.04it/s]


In [32]:
df = pt.get_dataframe().drop(["col_0", "col_1", "col_2"], axis=1)

In [33]:
df = df.set_index("potential")

In [34]:
df
#df.to_csv("potential_validation_Al.csv")

,job_id,a_eq,bulk_modulus,vac_formation,ecoh
potential,,,,,
Al_H_Ni_Angelo_eam,21,4.04995,81.0404,0.546216,-3.36
Al_Mg_Mendelev_eam,33,4.04542,89.0155,0.667786,-3.41066
Zope_Ti_Al_2003_eam,45,4.04995,80.8368,0.720309,-3.29877
Al_H_Ni_Angelo_eam,57,4.04995,81.0404,0.546216,-3.36
2000--Landa-A--Al-Pb--LAMMPS--ipr1,71,4.03125,78.2138,0.688258,-3.35928
2004--Zhou-X-W--Al--LAMMPS--ipr2,83,4.05032,71.5466,0.66806,-3.57998
2003--Zope-R-R--Al--LAMMPS--ipr1,95,4.04995,80.8368,0.720309,-3.35392
2015--Mendelev-M-I--Al-Sm--LAMMPS--ipr1,107,4.0412,85.0174,0.76364,-3.90515
1997--Liu-X-Y--Al-Mg--LAMMPS--ipr1,119,4.03266,83.4986,0.70411,-3.36005
